# Relatório Parcial Projeto de Supercomputação

## Aluno: Arthur Gomes Chieppe

Algortimos implementados:
* Gulosa
* Gulosa aleatória


## Etapa 1: Geração dos arquivos de input

### 1.1 Gerador automatizado

Para a geração dos arquivos de input do algoritmo, foi utilizado o código em C++ disponibilizado na página de introdução do projeto. Nenhum alteração foi feita, e o código encontra-se em /input_generator/gerador.cpp.

Foi desenvolvido um programa em Python responsável por rodar o gerador supracitado. Dessa forma, é possível especificar o número máximo de filmes e categorias que deseja-se gerar. O programa está localizado em `/input_generator/generate_inputs.py`, sendo que o C++ compilado tem de estar na mesma pasta. Os arquivos gerados são salvos em um novo diretório, em `/input_generator/input_files/`.O modo de uso do programa pode ser acessado com a flag --help, e está replicado abaixo:

```
python3 generate_inputs.py --help
usage: generate_inputs.py [-h] [--n_movies N_MOVIES] [--n_cat N_CAT]

Generate inputs for the model

options:
  -h, --help           show this help message and exit
  --n_movies N_MOVIES  Number of movies to generate
  --n_cat N_CAT        Number of categories to generate
```


Note que o gerador gerará os filmes respeitando um intervalo de 50 filmes entre os arquivos de inputs gerados.

## Etapa 2: A Heurística Gulosa

### 2.1 Definição de Escopo e Regras dos Algortimos

Foram feitas escolhas por questões de padronização e coerência dos resultados a serem obtidos. As regras abaixo são gerais, portanto valeu tanto para a heurística gulosa quanto a aleatória.

1. Não são admitidos filmes que tem datas de início e fim em dias diferentes. (Ex: início às 23 e fim às 2)
2. Não são admitidos filmes que começam e terminam no mesmo horário, pois isso implicaria em duração igual a 0. (Ex: início às 15 e fim às 15)

Os filmes que se enquadram em qualquer uma das situações acima são considerados inválidos, portanto não podem ser adicionados ao quadro de horários.


 ### 2.2 O algoritmo

 Inicialmente, implementou-se um algortimo com abordagem gulosa para a resolução do problema. De forma simplória, a abordagem escolhida baseia-se em:

 1. Ordenar os filmes por hora de fim crescente.
 2. Percorrer a lista de filmes ordenada, verificando se cabem no quadro de horários.
 3. Verificar se há vagas disponíveis para a categoria do filme em questão

Se forem satisfeitas as condições 2 e 3, o filme é adicionado ao quadro de horários.
 

### 2.3 Implementação em C++

A implementação do algoritmo está disponível em `heuristicas/gulosa/gulosa_filmes.cpp`. A compilação pode ser feita com o comando: `g++ gulosa_files.cpp -Wall -O3 -g -o gulosa` (assumindo que o usuário encontra-se na pasta do arquivo). Usuários de Apple Silicon podem compilar com `g++-12 gulosa_files.cpp -Wall -O3 -g -o gulosa`.

### 2.3.1 Varíaveis

1. `struct filme {int h_inicio; int h_fim; int cat; }`

* Struct que representa um filme, com horário de início, horário de fim e categoria a qual pertence, respectivamente.

2. `bool timetable[24]`
* Array de booleanos de tamanho 24, representando as horas do dia a serem ocupadas pelos filmes.
* Todos os valores são inicados como false.
* Obs: como pode ser constatado no código fonte, um filme que dure das 10 até às 11 ocupa apenas o timetable[10], de forma que seja possível que outro filme comece às 11.

3. `vector<int> categorias`
* Vetor de inteiros que representa as categorias dos filmes.
* Cada posição do vetor representa uma categoria, e o valor armazenado é a quantidade máxima de filmes que a categoria pode ter no quadro de horários.
* Ex: categorias[0] = 2, significa que a categoria 1 pode ter no máximo 2 filmes no quadro de horários. (o indice sempre é n-1)

4. `long unsigned int allowed_number_movies`

* Variável que representa o número máximo de filmes que podem ser adicionados ao quadro de horários. Obtido a partir da soma do número de filmes permitidos para cada categoria.
* Ex: se categorias[0] = 2 e categorias[1] = 3, então allowed_number_movies = 5. Dessa forma, o quadro de horários pode ter no máximo 5 filmes. Se atingido, o algoritmo para, significando uma solução ótima no que diz respeito ao número de filmes.
* Como não permite-se que um filme comece e termine no mesmo horário, o número máximo de filmes é sempre menor que o número de horas do dia, sendo nesse caso 23.

5. `vector<filmes> filmes`
* Vetor de structs que representa os filmes do arquivo de input.
* Cada posição do vetor representa um filme, e o valor armazenado é uma struct filme com os dados do filme.
* Ex: filmes[0] = {10, 11, 2}, significa que o filme 1 começa às 10 e termina às 11, e pertence à categoria 2.

6. `vector<filmes> programacao`
* Vetor de structs que representa os filmes que foram adicionados ao quadro de horários.
* Cada posição do vetor representa um filme, e o valor armazenado é uma struct filme com os dados do filme.
* Ex: programacao[0] = {10, 11, 2}, significa que o filme 1 começa às 10 e termina às 11, e pertence à categoria 2 foi adicionado.

7. `int n_filmes`
* Variável que representa o número de filmes do arquivo de input.
* Obtido a partir da primeira linha do arquivo de input.

8. `int n_cat`
* Variável que representa o número de categorias do arquivo de input.
* Obtido a partir da segunda linha do arquivo de input.

9. `int total_number_of_hours`
* Variável que representa o número total de horas preenchidas por filmes no quadro de horários.
* Obtido a partir da soma do tempo de todos os filmes adicionados ao quadro de horários.

### 2.3.2 Funções

1. `bool is_available(filme &el, bool (&timetable)[24], vector<int> &categorias)`
   * Verifica se um filme está disponível em uma determinada hora e categoria.
   * Recebe um objeto do tipo 'filme', um array de booleanos de tamanho 24 e um vetor de inteiros (categorias).
   * Retorna true se o filme está disponível, caso contrário, retorna false.

2. `void print_filmes(vector<filme> &filmes)`
   * Imprime na saída padrão o número total de filmes e a lista completa de filmes com suas respectivas informações (output gráfico).
   * Recebe um vetor de objetos do tipo 'filme'.
   * Não retorna nada.

3. `bool should_break(int &total_number_of_hours, long unsigned int &allowed_number_movies, vector<filme> &programacao)`
   * Verifica se o número total de horas de filmes exibidos ou o número máximo de filmes permitidos já foi alcançado.
   * Recebe um inteiro 'total_number_of_hours', um inteiro longo sem sinal 'allowed_number_movies' e um vetor de objetos do tipo 'filme'.
   * Retorna true se o número máximo de horas de filmes exibidos ou o número máximo de filmes permitidos foi alcançado, caso contrário, retorna false.

4. `void add_to_timetable(filme &el, bool (&timetable)[24], vector<int> &categorias, vector<filme> &programacao, int &total_number_of_hours)`
   * Adiciona um filme na tabela de horários e atualiza o número total de horas de filmes exibidos e o número total de filmes.
   * Recebe um objeto do tipo 'filme', um array de booleanos de tamanho 24, um vetor de inteiros, um vetor de objetos do tipo 'filme' e um inteiro 'total_number_of_hours'.
   * Não retorna nada.


### 2.3.3 Funcionamento do algoritmo

Após recebidos e armazenados os filmes do arquivo de input, o algoritmo começa então a ordernar os filmes por ordem crescente de horário de fim usando a função `sort`. Em seguida, percorre cada filme na ordem da lista, e verifica se ele pode ser exibido com ajuda da função `is_available`. Se o filme puder ser exibido, ele é adicionado ao quadro de horários e à programação com a função `add_to_timetable`. Caso contrário, o algoritmo passa para o próximo filme. O algoritmo para quando o número máximo de filmes permitidos ou o número máximo de horas de filmes exibidos é alcançado. Naturalmente, ele também para quando todos os filmes forem percorridos.

Por fim, o programa verifica se o número máximo de horas foi alcançado ou se todos os filmes já foram adicionados à programacao. Se alguma dessas condições for verdadeira, a execução do loop é interrompida e a função `print_filmes` é chamada para imprimir os filmes presentes na programacao. Se algum argumento de linha de comando foi fornecido, o número total de filmes e de horas é impresso na saída padrão. Esse recurso é utilizado para a realização dos testes de desempenho do algoritmo com um programa python, de forma que o tempo de execução do algoritmo não seja influenciado pela impressão dos resultados. O argumento a ser fornecido é qualquer string, como por exemplo "teste".